<a href="https://colab.research.google.com/github/Chirag-Kathuria-009/FEVER_FACT_CHECKER/blob/master/NLP_Fact_Checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install wikipedia
!pip install -U datasets
from datasets import load_dataset

# Load FEVER dataset which includes Wikipedia pages
#fever_dataset = load_dataset("fever", "wiki_pages",trust_remote_code=True)
fever_dataset = load_dataset('fever', 'wiki_pages', streaming=True,trust_remote_code=True)
# Load fever dataset for information about claims data
claims_data = load_dataset("fever", "v1.0",split='train',trust_remote_code=True)

claims_data_validation = load_dataset("fever", "v1.0",split='labelled_dev',trust_remote_code=True)
#claims_data_validation

Dataset({
    features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id'],
    num_rows: 37566
})

In [17]:

from itertools import groupby
distinct_sentenceids = {k for k, _ in groupby(sorted(d['evidence_wiki_url'] for d in claims_data))}
validation_sentenceids = {k for k, _ in groupby(sorted(d['evidence_wiki_url'] for d in claims_data_validation))}
distinct_sentenceids.update(validation_sentenceids)

print(distinct_sentenceids)
#print(claims_data[0])

{'', 'Mycroft_Holmes', 'Dave_Franco', 'Jessica_Jones_-LRB-TV_series-RRB-', 'Professional_wrestling_promotion', '2010_Davis_Cup', 'Twisted_Pictures', 'Galatasaray_S.K._-LRB-football-RRB-', 'Naples', '2003_NBA_draft', 'Like_Crazy', 'Kane_Jenkins', 'East_Palo_Alto,_California', 'Cyndi_Lauper', 'Animal_House', 'Jackie_Evancho', 'Lions_for_Lambs', 'United_Nations_Development_Fund_for_Women', 'Sicario_-LRB-2015_film-RRB-', 'The_Great_Outdoors_-LRB-film-RRB-', 'Poison_Ivy_-LRB-1992_film-RRB-', 'Loki_-LRB-comics-RRB-', 'Newfoundland_-LRB-island-RRB-', "The_Dogs_D'Amour", 'YouTube', 'Granite', 'Duane_Hopwood', 'Time_Person_of_the_Year', 'Bengal', 'Chagatai_language', 'Cap_-LRB-sport-RRB-', 'Simon_&_Garfunkel', 'Roman_Republic', 'To_Die_For', 'Houston_Rockets', 'Video_Games_-LRB-song-RRB-', 'Janet_Leigh', 'Anna_Kendrick', 'Batgirl', 'Damages_-LRB-TV_series-RRB-', 'Joshua_Logan', 'Beethoven_-LRB-film-RRB-', 'Tha_Last_Meal', 'Université_Laval', 'Internment_of_Japanese_Americans', 'An_American_Rha

In [ ]:
#distinct_sentenceids.remove('')

import tqdm
import unicodedata

pages = fever_dataset['wikipedia_pages']
filtered_pages = []
for page in tqdm.tqdm(pages):
  page_id = str(page.get("id"))
  if page_id in distinct_sentenceids and page not in filtered_pages:
    filtered_pages.append(page)

  if len(filtered_pages) == len(distinct_sentenceids):
        break

print(len(filtered_pages))
#Remind me of the roadmap for my fact checker project.”



In [ ]:
print(len(filtered_pages))
#print(len(distinct_sentenceids))
#print(filtered_pages[:10])
from google.colab import drive
import pickle
drive.mount('/content/drive')


# Assuming `filtered_pages` is your list
'''with open("/content/drive/MyDrive/filtered_pages.pkl", "wb") as f:
    pickle.dump(filtered_pages, f)
print("Saved filtered_pages to filtered_pages.pkl")'''

with open("/content/drive/MyDrive/filtered_pages.pkl", "rb") as f:
    filtered_pages = pickle.load(f)
print("Loaded filtered_pages from filtered_pages.pkl")
#print(d[0])
print(claims_data[:10])
print(len(filtered_pages))

In [4]:
def indexing_wiki_pages():
  wiki_index = {}
  error_pages = set()
  for page in filtered_pages:
    page_id = page['id']
    lines = page['lines'].split('\n')
    wiki_index[page_id] = {}
    for line in lines:
      if line.strip():
        try:
          sid, sentence = line.split("\t",1)
          wiki_index[page_id][int(sid)] = sentence

        except ValueError as error:
          error_pages.add(page_id)
          continue

  return wiki_index




In [5]:
from datasets import Dataset
def mapping_wiki_pages_sentence():
  indexed_sentence_pages = indexing_wiki_pages()
  missing_keys = set()
  corr_sentences = []
  for claim in claims_data:
    page_id = claim['evidence_wiki_url']
    sentence_id = claim['evidence_sentence_id']
    sentence=''
    if(sentence_id !=-1):

      try:
        value = indexed_sentence_pages.get(page_id)
        if value is not None:
          sentence = indexed_sentence_pages[page_id][sentence_id]
          #claim['corr_sentence'] = sentence
          #print('Sentences',claim['corr_sentence'])
        else:
          missing_keys.add(page_id)


      except ValueError as error:
        sentence = None
        print('Error with page_id',page_id+str(error))
        continue
    corr_sentences.append(sentence)

  if len(corr_sentences) != len(claims_data):
    print(f"Warning: Length mismatch! corr_sentences length: {len(corr_sentences)} vs claims_data length: {len(claims_data)}")

  if isinstance(claims_data, Dataset):
    updated_claims_data = claims_data.add_column('corr_sentence', corr_sentences)
  else:
    print("Error: claims_data is not of type datasets.Dataset")
  return updated_claims_data,missing_keys

#print(len(indexed_sentence_pages))



In [ ]:
updated_claims_sentences,keys_issue = mapping_wiki_pages_sentence()

print(updated_claims_sentences[:5])

#print(indexed_sentence_pages['Chris_Pérez'])


In [7]:
#print(type(claims_data)) Checking duplicate records in claims dataset
import pandas as pd
df = pd.DataFrame(updated_claims_sentences)
#df.head(10)



In [ ]:
# Just for understanding data whats actually going on
valid_ids = df[df['label'].isin(['SUPPORTS', 'REFUTES'])].groupby('id').filter(
    lambda g: (g['evidence_sentence_id'].eq(-1).any()) &
              (g['evidence_sentence_id'].nunique() > 1)
)['id'].unique()

# Then filter the original dataframe
result = df[(df['label'].isin(['SUPPORTS', 'REFUTES'])) &
            (df['id'].isin(valid_ids))].sort_values('id')
#result.head(10)

##   Above cases are those in which one is having claim as supported with evidence and other is having claim without evidence_id

results = df.groupby('id').filter(
    lambda group: (
        group['label'].isin(['SUPPORTS', 'REFUTES']).any() and
        (group['evidence_sentence_id'] == -1).any() and
        not (group['evidence_sentence_id'] != -1).any()
    )
)

'''
# one time check if logic works or not
result1 =  filtered_df.groupby('id').filter(
    lambda group: (
        group['label'].isin(['SUPPORTS', 'REFUTE']).any() and
        (group['evidence_sentence_id'] == -1).any() and
        not (group['evidence_sentence_id'] != -1).any()
    )
)
'''

common_ids = result['id'].isin(results['id']).any()
print(common_ids)
#results
#print(df[df['id']==129629])

In [ ]:
# Since there are no commonids we need to remove results id from main dataset because they are bit useless

issue_ids = results['id'].unique()
filtered_df = df[~df['id'].isin(issue_ids)]

filtered_df.head(10)

In [ ]:
aggregated_df = filtered_df.groupby(['id','label','claim'],as_index=False)['corr_sentence'].apply(' '.join)
aggregated_df.head(10)


condition = (
    aggregated_df['label'].isin(['SUPPORTS', 'REFUTES']) &
    (aggregated_df['corr_sentence'] == '')
)
matching_records = aggregated_df[condition]
df[df['id'].isin(matching_records['id']) & ~df['evidence_wiki_url'].isin(keys_issue)]

In [ ]:
# time being removing records with keys_issue

special_keys_issue = matching_records['id'].unique()

aggregated_df = aggregated_df[~aggregated_df['id'].isin(special_keys_issue)]


aggregated_df.size





In [12]:
def tokenize(example, type):
    if(type=='short'):

      return tokenizer(
          example["claim"],
          example["corr_sentence"],
          truncation=True,
          padding="max_length",
          max_length=512
      )
    else:
      chunks = tokenizer(example['claim'],
                         example['corr_sentence'],
                         truncation=True,
                         max_length = 512,
                         stride = 128,
                         return_overflowing_tokens = True,
                         padding = 'max_length')
      return chunks




In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

MAX_LEN = 512
# converting text labels into encoding for model understanding
LABEL_CODING = {'SUPPORTS':0,'REFUTES':1,'NOT ENOUGH INFO':2}
# example is like df
def handling_tokenisation(example):
  tokens = tokenizer(example['claim'],example['corr_sentence'], truncation = False)


  if len(tokens['input_ids'])<=MAX_LEN:
    res =  tokenize(example,'short')
    return [{
        'input_ids': res['input_ids'],
        'attention_mask': res['attention_mask'],
        'label_value' : LABEL_CODING[example['label']]
    }]



  else:
    res = tokenize(example,'long')
    # in this case chunks will be returned
    output = []
    for i in range(len(res['input_ids'])):
      output.append({

          'input_ids': res['input_ids'][i],
          'attention_mask': res['attention_mask'][i],
          'label_value' : LABEL_CODING[example['label']]
      })

    return output



In [ ]:
# applying tokenisation for entire dataframe
tokenized_data = []

for _, row in aggregated_df.iterrows():
  samples = handling_tokenisation(row)
  tokenized_data.extend(samples)


In [ ]:
claims_data_validation

In [ ]:
your_data_list = aggregated_df.to_dict(orient="records")
your_data_list

In [ ]:
# One time handling for special character situation
count=0
import unicodedata
import tqdm
#import unicodedata
special_pageids = []
s = 'Mélanie_Laurent'
s = unicodedata.normalize('NFC', s.strip().lower())
#print(s)
#print(filtered_pages[:10])
#print(list(distinct_sentenceids)[:10])
'''found_ids = [v['id'] for v in d]
for page_id in distinct_sentenceids:
  if page_id not in found_ids:

    special_pageids.append(unicodedata.normalize('NFC', page_id.strip().lower()))


pages = fever_dataset['wikipedia_pages']
# handling part
for page in tqdm.tqdm(pages):
  page_id = str(page.get("id"))
  page_id = unicodedata.normalize('NFC', page_id.strip().lower())
  if page_id in special_pageids and page not in d:
    d.append(page)

  if len(d) == len(distinct_sentenceids):
        break'''

#print(count)
#print(len(d))
print(len(filtered_pages))
print(len(distinct_sentenceids))
print(keys_issue)
print(claims_data[:10])
print(list(distinct_sentenceids).index('Alexander_Skarsgård'))
found_ids = [v['id'] for v in filtered_pages]
print(len(set(found_ids)))

print(type(claims_data))
print(claims_data[0])

def contains_non_english(word):
    # Check if any character in the word is a non-ASCII character
    return any(unicodedata.category(c) in ['Mn', 'Mc', 'Me'] for c in word)
non_english_words = [word for word in distinct_sentenceids if contains_non_english(word)]
print(len(non_english_words))